# Data Analysis and Embeddings for ABB Chatbot


In [ ]:
%pip install pandas openpyxl sentence-transformers



In [ ]:
import pandas as pd

file1 = "/home/unsettledaverage73/ABB-chatbot/abb-chatbot/3439_RMU_StockObso_2507A-1(1).csv"
file2 = "/home/unsettledaverage73/ABB-chatbot/abb-chatbot/sustainbility2(1).csv"

print(f"--- Analyzing {file1} ---")
try:
    df1 = pd.read_excel(file1)
    print("Head of the dataframe:")
    print(df1.head())
    print("\nColumns:")
    print(df1.columns)
    print("\nSummary statistics:")
    print(df1.describe())
except Exception as e:
    print(f"Error reading {file1}: {e}")

print(f"\n--- Analyzing {file2} ---")
try:
    df2 = pd.read_excel(file2)
    print("Head of the dataframe:")
    print(df2.head())
    print("\nColumns:")
    print(df2.columns)
    print("\nSummary statistics:")
    print(df2.describe())
except Exception as e:
    print(f"Error reading {file2}: {e}")


In [ ]:
from sentence_transformers import SentenceTransformer

# Initialize the Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Prepare text for embeddings for df1 (3439_RMU_StockObso_2507A-1(1).csv)
# Convert all relevant columns to string and join them
if 'df1' in locals() and not df1.empty:
    df1_text_columns = ['Manual', 'Jan 2025', 'Dec 2024']
    df1['combined_text'] = df1[df1_text_columns].astype(str).agg(' '.join, axis=1)
    df1['embeddings'] = list(model.encode(df1['combined_text'].tolist()))
    print(f"\nEmbeddings generated for {file1}. Shape: {df1['embeddings'].shape}")
    print("First 5 embeddings for df1:")
    for i, embedding in enumerate(df1['embeddings'].head()):
        print(f"Row {i}: {embedding[:5]}...") # print first 5 dimensions of embedding
else:
    print(f"\n{file1} is empty or not loaded, skipping embeddings.")

# Prepare text for embeddings for df2 (sustainbility2(1).csv)
# Convert all relevant columns to string and join them
if 'df2' in locals() and not df2.empty:
    df2_text_columns = [
        'Date', 'Plastic Reduced (kg)', 'Daily Plastic Consumption (kg)',
        'Wood Consumption (kg)', 'Energy Consumption (kWh)', 'E-Waste (kg)',
        'SF6 Consumption (kg)', 'Argon Consumption (kg)', 'Helium Consumption (kg)',
        'CO2 Emission (kg)', 'Hazardous Waste (kg)'
    ]
    # Ensure all columns exist before trying to combine them
    existing_df2_text_columns = [col for col in df2_text_columns if col in df2.columns]
    if existing_df2_text_columns:
        df2['combined_text'] = df2[existing_df2_text_columns].astype(str).agg(' '.join, axis=1)
        df2['embeddings'] = list(model.encode(df2['combined_text'].tolist()))
        print(f"\nEmbeddings generated for {file2}. Shape: {df2['embeddings'].shape}")
        print("First 5 embeddings for df2:")
        for i, embedding in enumerate(df2['embeddings'].head()):
            print(f"Row {i}: {embedding[:5]}...") # print first 5 dimensions of embedding
    else:
        print(f"\nNo relevant text columns found in {file2}, skipping embeddings.")
else:
    print(f"\n{file2} is empty or not loaded, skipping embeddings.")


In [ ]:
import faiss
import numpy as np

# Convert embeddings to a contiguous NumPy array with float32 type for FAISS
if 'df2' in locals() and 'embeddings' in df2.columns and not df2['embeddings'].empty:
    embeddings_array = np.array(df2['embeddings'].tolist()).astype('float32')

    # Initialize a FAISS index
    # Using IndexFlatL2 for a simple L2 distance (Euclidean distance) index
    dimension = embeddings_array.shape[1]
    index = faiss.IndexFlatL2(dimension)

    # Add the embeddings to the index
    index.add(embeddings_array)

    print(f"\nFAISS index created with {index.ntotal} embeddings.")

    # Example: Query the vector store
    query_text = "What is the plastic consumption?"
    query_embedding = model.encode([query_text]).astype('float32')

    D, I = index.search(query_embedding, k=3)  # Search for the 3 most similar embeddings

    print(f"\nQuery: {query_text}")
    print("Most similar entries (FAISS scores and indices):")
    for i in range(len(I[0])):
        print(f"  Score: {D[0][i]:.4f}, Index: {I[0][i]}, Text: {df2.loc[I[0][i], 'combined_text']}")
else:
    print(f"\nDataFrame df2 or its embeddings are not available, skipping FAISS index creation and query.")


## 4. Generation: Answering Questions with an LLM


In [ ]:
from transformers import pipeline

# Initialize a question-answering pipeline with an open-source LLM
# We'll use distilbert-base-uncased-distilled-squad for better extractive QA performance
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

# Assuming 'query_text' and 'I' (indices of relevant documents) are available from the previous step
# We'll re-define them here for standalone execution of this cell if needed.
# In a full run, these would be passed from the previous cell's execution.
if 'query_text' not in locals():
    query_text = "What is the plastic consumption?"

if 'I' in locals() and 'df2' in locals():
    # Get the most relevant context from df2 based on the FAISS search results
    # Concatenate the 'combined_text' from the top retrieved rows
    retrieved_contexts = [df2.loc[idx, 'combined_text'] for idx in I[0]]
    context = " ".join(retrieved_contexts)

    print(f"\nOriginal Question: {query_text}")
    print(f"Retrieved Context (from relevant CSV rows):\n{context[:500]}...") # Print first 500 chars of context

    # Use the LLM to generate an answer
    # We instruct the model to only use the provided context
    llm_response = qa_pipeline(question=query_text, context=context)

    print(f"\nLLM Answer: {llm_response['answer']}")
    print(f"Confidence Score: {llm_response['score']:.4f}")
else:
    print("\nPrevious steps for query and retrieval were not executed. Please run prior cells.")
